# Multi-tenant Chat with Papers - Query papers
## Get keys and urls

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")

print(f"Weaviate Key:{WEAVIATE_KEY}")
print(f"OpenAI API Key: {OPENAI_API_KEY[:20]}")
print(f"OpenAI URL: {OPENAI_URL}")

## Connect to Weaviate

In [ ]:
import weaviate
from weaviate.classes.init import Auth

# Connect to the local instance
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,
  grpc_port=50051,
  auth_credentials=Auth.api_key(WEAVIATE_KEY),
  headers={
    "X-OpenAI-Api-Key": OPENAI_API_KEY
  }
)

print(client.is_ready())

## Vector search on tenants

In [ ]:
papers = client.collections.get("Papers")

ten = papers.with_tenant("2212-10496")

response = ten.query.near_text(
    query="Unsupervised learning",
    limit=5,
)

for item in response.objects:
    print(item.properties["chunk"])

## Generative Search with tenants

In [ ]:
papers = client.collections.get("Papers")

ten2212 = papers.with_tenant("2212-10496")

response = ten2212.generate.near_text(
    query="Unsupervised learning",
    limit=3,
    single_prompt="What does the following text describe: {chunk}",
)

for item in response.objects:
    print(item.properties["chunk"])
    print(item.generative.text, '\n')

In [ ]:
papers = client.collections.get("Papers")

ten2212 = papers.with_tenant("2212-10496")

response = ten2212.generate.near_text(
    query="Unsupervised learning",
    limit=3,
    grouped_task="Explain how unsupervised learning works. Use only the provided content.",
    grouped_properties=["chunk"]
)

for item in response.objects:
    print(item.properties["chunk"])

print(response.generative.text)

In [ ]:
def paper_rag(paper_id, query, prompt):
    papers = client.collections.get("Papers")
    ten = papers.with_tenant(paper_id)

    response = ten.generate.near_text(
        query=query,
        limit=3,
        grouped_task=prompt + " Use only the provided content.",
        grouped_properties=["chunk"],
    )

    return {
        "title": response.objects[0].properties["title"],
        "source": [p.properties["chunk"] for p in response.objects],
        "generated": response.generative.text
    }

In [ ]:
paper_rag(
    "2212-10496",
    "Unsupervised learning",
    "Explain how unsupervised learning works"
)

In [ ]:
papers = client.collections.get("Papers")
papers.tenants.get()

## Close the client

In [ ]:
client.close()